In [1]:
#same as prep with some holidays merged

In [2]:
import numpy as np
import pandas as pd

In [3]:
train=pd.read_csv(r'originalni_datasetovi\train.csv',parse_dates=['date'])
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [5]:
train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [6]:
test=pd.read_csv(r'originalni_datasetovi\test.csv', parse_dates=['date'])
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [7]:
#reading other datasets
holidays=pd.read_csv(r'originalni_datasetovi\holidays_events.csv',parse_dates=['date'])
oil=pd.read_csv(r'originalni_datasetovi\oil.csv', parse_dates=['date'])
stores=pd.read_csv(r'originalni_datasetovi\stores.csv')
transactions=pd.read_csv(r'originalni_datasetovi\transactions.csv', parse_dates=['date'])

In [8]:
#check for missing dates in train
date_min_train=train.date.min()
date_max_train=train.date.max()
ndays_train=(date_max_train-date_min_train).days+1 
print("Number of days that should be in train: ",ndays_train)
actdays_train=train.date.nunique()
print("Number of days that are in train: ",actdays_train)
print("Missing days: ", ndays_train-actdays_train)

Number of days that should be in train:  1688
Number of days that are in train:  1684
Missing days:  4


In [9]:
#inspecting missing dates in train
missing_dates = pd.date_range(date_min_train, date_max_train).difference(train.date.unique()).tolist()
missing_dates

[Timestamp('2013-12-25 00:00:00'),
 Timestamp('2014-12-25 00:00:00'),
 Timestamp('2015-12-25 00:00:00'),
 Timestamp('2016-12-25 00:00:00')]

In [10]:
#check for missing dates in test
date_min_test=test.date.min()
date_max_test=test.date.max()
ndays_test=(date_max_test-date_min_test).days+1 
print("Number of days that should be in train: ",ndays_test)
actdays_test=test.date.nunique()
print("Number of days that are in train: ",actdays_test)
print("Missing days: ", ndays_test-actdays_test)

Number of days that should be in train:  16
Number of days that are in train:  16
Missing days:  0


In [11]:
#solving missing train dates

#makes multiindex with all combination of dates, stores and families (has missing dates)
multi_index=pd.MultiIndex.from_product(
    [pd.date_range(date_min_train,date_max_train), train.store_nbr.unique(), train.family.unique()],
    names=['date','store_nbr','family'])

#first, making MultiIndex with existing dates, stores and families
train=train.set_index(['date','store_nbr','family'])

#then, reindexing with multi_index, on places where old index (missing 4 dates) didn't exist are NaN values
train=train.reindex(multi_index)

#reseting so that the index levels become regular columns again
train=train.reset_index()

#filling missing values with zeros
train[['sales','onpromotion']]=train[['sales','onpromotion']].fillna(0)

In [12]:
#making new id column
train['id']=np.arange(0,train.shape[0])
#place it as a first column
train=train[['id']+[col for col in train.columns if col!='id']]

train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0


In [13]:
NO_FAMILIES=train.family.nunique()
NO_STORES=train.store_nbr.nunique()

In [14]:
#dealing with holidays

#TYPE: transferred - not acc celebrated on that day

transferred_holidays = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True) #holidays that were transferred
transfer = holidays[(holidays.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True) #days that they were transferred to (in ds they are always below the actual holiday so indexes are going to match)
tr = pd.concat([transferred_holidays,transfer], axis = 1) 
tr = tr.iloc[:, [5,1,2,3,4]] #getting rid of dupplicate columns, keeping date from transferred, everything else from transferred_holidays
tr

,date,type,locale,locale_name,description
0,2012-10-12,Holiday,National,Ecuador,Independencia de Guayaquil
1,2013-10-11,Holiday,National,Ecuador,Independencia de Guayaquil
2,2014-10-10,Holiday,National,Ecuador,Independencia de Guayaquil
3,2016-05-27,Holiday,National,Ecuador,Batalla de Pichincha
4,2016-07-24,Holiday,Local,Guayaquil,Fundacion de Guayaquil
5,2016-08-12,Holiday,National,Ecuador,Primer Grito de Independencia
6,2017-01-02,Holiday,National,Ecuador,Primer dia del ano
7,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca
8,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha
9,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia


In [15]:
#deleting transferred holidays and transfer days from holidays and we don't need transferred column anymore
holidays=holidays[(holidays.transferred==False) & (holidays.type !='Transfer')].drop('transferred',axis=1)

#adding concated transferred holidays and transfer days
holidays=pd.concat([holidays,tr]).reset_index(drop=True)

holidays

,date,type,locale,locale_name,description
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba
...,...,...,...,...,...
333,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca
334,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha
335,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia
336,2017-09-29,Holiday,Local,Ibarra,Fundacion de Ibarra


In [16]:
import re
#TYPE: Additional 

#have the same name as regular holiday but with + or - number 
#removing +-number
def clean_description(desc):
    desc = re.sub(r'[+-]', '', desc)  # Remove + and -
    desc = re.sub(r'\d+', '', desc)  # Remove digits
    return desc

holidays["description"] = holidays["description"].apply(clean_description)

#TYPE: Bridge

#have the same name as regular holiday but with Puente at the beginning
#removing puente
holidays["description"] = holidays["description"].str.replace("Puente ", "")

In [17]:
#TYPE: Work Day

#work days are not actually holidays, they are just days that you wouldn't regularly work on but now you do (saturdays)
#removing them
work_days=holidays[holidays['type']=='Work Day']
holidays=holidays[holidays['type']!='Work Day']

In [18]:
#TYPE: Event

#"There are many football events referred to by the match name that include the word 'futbal'
holidays.loc[holidays["description"].str.contains("futbol"), "description"] = "Futbol"

In [19]:
pd.set_option('display.max_rows', None)
holidays

,date,type,locale,locale_name,description
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala


In [20]:
#Splitting by locale

#Local holidays - city level
local_holidays=holidays[holidays['locale']=='Local']
local_holidays=local_holidays.rename(columns={'locale_name':'city','description': 'local_holidays'})
local_holidays=local_holidays.reset_index(drop=True)
local_holidays.drop(columns=['locale','type'],inplace=True)

#Regional holidays
regional_holidays=holidays[holidays['locale']=='Regional']
regional_holidays=regional_holidays.rename(columns={'locale_name':'state', 'description':'regional_holidays'})
regional_holidays=regional_holidays.reset_index(drop=True)
regional_holidays.drop(columns=['locale','type'],inplace=True)

#National holidays adn events
national_holidays=holidays[holidays['locale']=='National']
national_holidays=national_holidays.rename(columns={'description':'national_holidays'})
national_holidays=national_holidays.reset_index(drop=True)
national_holidays.drop(columns=['locale','type','locale_name'],inplace=True)

In [25]:
#there are 3 main types of local holidays Fundacion , Cantonizacion and Independencia 
local_holidays.loc[local_holidays["local_holidays"].str.contains("Fundacion"), "local_holidays"] = "Fundacion"
local_holidays.loc[local_holidays["local_holidays"].str.contains("Cantonizacion"), "local_holidays"] = "Cantonizacion"
local_holidays.loc[local_holidays["local_holidays"].str.contains("Independencia"), "local_holidays"] = "Independencia"
local_holidays.head(20)

,date,city,local_holidays
0,2012-03-02,Manta,Fundacion
1,2012-04-12,Cuenca,Fundacion
2,2012-04-14,Libertad,Cantonizacion
3,2012-04-21,Riobamba,Cantonizacion
4,2012-05-12,Puyo,Cantonizacion
5,2012-06-23,Guaranda,Cantonizacion
6,2012-06-25,Latacunga,Cantonizacion
7,2012-06-25,Machala,Fundacion
8,2012-07-03,Santo Domingo,Fundacion
9,2012-07-03,El Carmen,Cantonizacion


In [27]:
#all regional holidays are Provincializacion
regional_holidays.loc[regional_holidays["regional_holidays"].str.contains("Provincializacion"), "regional_holidays"] = "Provincializacion"
regional_holidays.head(50)

,date,state,regional_holidays
0,2012-04-01,Cotopaxi,Provincializacion
1,2012-06-25,Imbabura,Provincializacion
2,2012-11-06,Santo Domingo de los Tsachilas,Provincializacion
3,2012-11-07,Santa Elena,Provincializacion
4,2013-04-01,Cotopaxi,Provincializacion
5,2013-06-25,Imbabura,Provincializacion
6,2013-11-06,Santo Domingo de los Tsachilas,Provincializacion
7,2013-11-07,Santa Elena,Provincializacion
8,2014-04-01,Cotopaxi,Provincializacion
9,2014-06-25,Imbabura,Provincializacion


In [28]:
national_holidays.national_holidays.unique() #potential clustering

array(['Primer Grito de Independencia', 'Dia de Difuntos',
       'Independencia de Cuenca', 'Navidad', 'Primer dia del ano',
       'Carnaval', 'Viernes Santo', 'Dia del Trabajo', 'Dia de la Madre',
       'Batalla de Pichincha', 'Futbol', 'Black Friday', 'Cyber Monday',
       'Independencia de Guayaquil', 'Terremoto Manabi'], dtype=object)

In [29]:
#concating train and test to add additional columns
df = pd.concat([train, test]).reset_index(drop=True)
df['id']=np.arange(0,df.shape[0])

In [30]:
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0


In [31]:
df.tail()

,id,date,store_nbr,family,sales,onpromotion
3036523,3036523,2017-08-31,9,POULTRY,NaN,1.0
3036524,3036524,2017-08-31,9,PREPARED FOODS,NaN,0.0
3036525,3036525,2017-08-31,9,PRODUCE,NaN,1.0
3036526,3036526,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0
3036527,3036527,2017-08-31,9,SEAFOOD,NaN,0.0


In [32]:
#Adding stores data
df=pd.merge(df,stores)
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13


In [33]:
df.shape[0]

3036528

In [34]:
#Adding holidays data

#work days
work_days=work_days[['date','type']].rename(columns={'type':'work_day'}).reset_index(drop=True)
work_days.work_day=work_days.work_day.notna().astype(int)
df=pd.merge(df,work_days, how='left', on='date')
df['work_day']=df['work_day'].fillna(0).astype(int)
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0


In [38]:
#local holidays - adding 3 columns for Fundacion , Cantonizacion and Independencia 
#there could be multiple holidays on the same day so one hot encoding doesn't work
local_holidays_wide = local_holidays.pivot_table(index=['date','city'], columns=['local_holidays'], aggfunc='size', fill_value=0)
local_holidays_wide = local_holidays_wide.astype(int)
local_holidays_wide = local_holidays_wide.reset_index() #to return date as a column
local_holidays_wide.head()

local_holidays,date,city,Cantonizacion,Fundacion,Independencia
0,2012-03-02,Manta,0,1,0
1,2012-04-12,Cuenca,0,1,0
2,2012-04-14,Libertad,1,0,0
3,2012-04-21,Riobamba,1,0,0
4,2012-05-12,Puyo,1,0,0


In [44]:
#merging 3 new columns with df and filling NaN with zeros
df=pd.merge(df,local_holidays_wide, how='left', on=['date', 'city'])
for col in local_holidays_wide.columns:
    if col not in ['date', 'city']:
        df[col] = df[col].fillna(0).astype(int)
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,Cantonizacion,Fundacion,Independencia
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0


In [54]:
#regional holidays
regional_holidays_wide = regional_holidays.pivot_table(index=['date','state'], columns=['regional_holidays'], aggfunc='size', fill_value=0)
regional_holidays_wide = regional_holidays_wide.astype(int)
regional_holidays_wide = regional_holidays_wide.reset_index() #to return date as a column
regional_holidays_wide.head()

regional_holidays,date,state,Provincializacion
0,2012-04-01,Cotopaxi,1
1,2012-06-25,Imbabura,1
2,2012-11-06,Santo Domingo de los Tsachilas,1
3,2012-11-07,Santa Elena,1
4,2013-04-01,Cotopaxi,1


In [55]:
#merging new column and filling NaN with zeros
df=pd.merge(df,regional_holidays_wide, how='left', on=['date','state'])
for col in regional_holidays_wide.columns:
    if col not in ['date', 'state']:
        df[col] = df[col].fillna(0).astype(int)
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,Cantonizacion,Fundacion,Independencia,Provincializacion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0


In [56]:
#national holidays
national_holidays_wide = national_holidays.pivot_table(index='date', columns='national_holidays', aggfunc='size', fill_value=0)
national_holidays_wide = national_holidays_wide.astype(int)
national_holidays_wide = national_holidays_wide.reset_index() #to return date as a column
df=pd.merge(df,national_holidays_wide, how='left', on=['date'])
for col in national_holidays_wide.columns:
    if col != 'date':
        df[col] = df[col].fillna(0).astype(int)
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,Dia de la Madre,Dia del Trabajo,Futbol,Independencia de Cuenca,Independencia de Guayaquil,Navidad,Primer Grito de Independencia,Primer dia del ano,Terremoto Manabi,Viernes Santo
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,1,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,1,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,1,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,1,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,1,0,0


In [57]:
df.shape[0] #Check if the number of rows changed

3036528

In [58]:
#oil data
import datetime

#checking for missing dates in dataset
print('Missing oil dates: ',df.shape[0]/NO_STORES/NO_FAMILIES-oil.shape[0])
missing_oil_dates=pd.date_range(date_min_train, date_max_test).difference(oil.date)
missing_oil_days=[]
for date in missing_oil_dates:
    day_name = datetime.date.strftime(date, '%A')
    missing_oil_days.append(day_name)

print(set(missing_oil_days)) #checking are oil prices missing just on weekends

Missing oil dates:  486.0
{'Sunday', 'Saturday'}


In [59]:
oil.rename(columns={'dcoilwtico':'oil_price'},inplace=True)

In [60]:
#checking for NaN values 
oil.isna().sum()

date          0
oil_price    43
dtype: int64

In [61]:
#adding missing dates and reindexing data
oil = oil.merge(
    pd.DataFrame({"date": pd.date_range(date_min_train, date_max_test)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

In [62]:
#filling missing values

oil['oil_price'] = oil['oil_price'].interpolate(method='linear')
oil['oil_price'] = oil['oil_price'].fillna(method='bfill') #for the first missing value
print('Number of missing values:', oil['oil_price'].isna().sum())

Number of missing values: 0


In [63]:
oil.head(10)

,date,oil_price
0,2013-01-01,93.140000
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.146667
5,2013-01-06,93.173333
6,2013-01-07,93.200000
7,2013-01-08,93.210000
8,2013-01-09,93.080000
9,2013-01-10,93.810000


In [64]:
#adding oil data
df=pd.merge(df,oil,how='left',on='date')
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,Dia del Trabajo,Futbol,Independencia de Cuenca,Independencia de Guayaquil,Navidad,Primer Grito de Independencia,Primer dia del ano,Terremoto Manabi,Viernes Santo,oil_price
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1,0,0,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1,0,0,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1,0,0,93.14
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1,0,0,93.14
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1,0,0,93.14


In [65]:
#transactions data

#checking for missing dates
missing_transactions_dates=pd.date_range(date_min_train, date_max_test).difference(transactions.date)
#dates that don't have entries for all 54 stores
store_counts = transactions.groupby('date').size()
missing_stores_dates = store_counts[store_counts < 54].index.tolist()

print("Missing dates")
print(missing_transactions_dates)
print(len(missing_transactions_dates))

print("Dates that don't have all entries")
print(missing_stores_dates)
print(len(missing_stores_dates))

Missing dates
DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-01-01',
               '2016-01-03', '2016-12-25', '2017-08-16', '2017-08-17',
               '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21',
               '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25',
               '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
               '2017-08-30', '2017-08-31'],
              dtype='datetime64[ns]', freq=None)
22
Dates that don't have all entries
[Timestamp('2013-01-01 00:00:00'), Timestamp('2013-01-02 00:00:00'), Timestamp('2013-01-03 00:00:00'), Timestamp('2013-01-04 00:00:00'), Timestamp('2013-01-05 00:00:00'), Timestamp('2013-01-06 00:00:00'), Timestamp('2013-01-07 00:00:00'), Timestamp('2013-01-08 00:00:00'), Timestamp('2013-01-09 00:00:00'), Timestamp('2013-01-10 00:00:00'), Timestamp('2013-01-11 00:00:00'), Timestamp('2013-01-12 00:00:00'), Timestamp('2013-01-13 00:00:00'), Timestamp('2013-01-14 00:00:00'), Timestamp('2013-01-15 00

In [66]:
# list to hold missing stores per date
missing_stores = []

for date in missing_stores_dates:
    existing_stores = transactions[transactions['date'] == date]['store_nbr'].tolist()
    missing_stores_for_date = list(set(range(1, 55)) - set(existing_stores))
    for store in missing_stores_for_date:
        missing_stores.append({"date": date, "store_nbr": store})

missing_stores_df = pd.DataFrame(missing_stores)
missing_stores_df.head(10)

,date,store_nbr
0,2013-01-01,1
1,2013-01-01,2
2,2013-01-01,3
3,2013-01-01,4
4,2013-01-01,5
5,2013-01-01,6
6,2013-01-01,7
7,2013-01-01,8
8,2013-01-01,9
9,2013-01-01,10


In [67]:
#checking for NaN values 
transactions.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [68]:
#merge missing_stores_df with transactions to fill missing entries
transactions = pd.concat([transactions, missing_stores_df], ignore_index=True).sort_values(["date", "store_nbr"])
transactions=transactions.reset_index(drop=True)
transactions=transactions.fillna(0)
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,1,0.0
1,2013-01-01,2,0.0
2,2013-01-01,3,0.0
3,2013-01-01,4,0.0
4,2013-01-01,5,0.0


In [69]:
#adding missing dates and reindexing data
date_range_df = pd.DataFrame({"date": pd.date_range(date_min_train, date_max_test)})

# create a df with store numbers from 1 to 54 for each date
store_numbers_df = pd.DataFrame({
    "store_nbr": list(range(1, 55)) * len(date_range_df),
    "date": sorted(list(date_range_df['date']) * 54)
})
store_numbers_df=store_numbers_df[store_numbers_df['date'].isin(missing_transactions_dates)]

transactions = transactions.merge(store_numbers_df, on=["date","store_nbr"], how="outer")

In [70]:
#dates that had zero transactions
sum_of_sales=df[df['date'].isin(missing_transactions_dates)].groupby('date')['sales'].sum()
zero_transaction_dates=sum_of_sales[sum_of_sales==0].index.tolist()

#filling transactions with zeros on those dates
for date in zero_transaction_dates:
    print(date)
    transactions.loc[transactions['date']==date, 'transactions']=0

transactions.isna().sum()

2013-12-25 00:00:00
2014-12-25 00:00:00
2015-12-25 00:00:00
2016-12-25 00:00:00
2017-08-16 00:00:00
2017-08-17 00:00:00
2017-08-18 00:00:00
2017-08-19 00:00:00
2017-08-20 00:00:00
2017-08-21 00:00:00
2017-08-22 00:00:00
2017-08-23 00:00:00
2017-08-24 00:00:00
2017-08-25 00:00:00
2017-08-26 00:00:00
2017-08-27 00:00:00
2017-08-28 00:00:00
2017-08-29 00:00:00
2017-08-30 00:00:00
2017-08-31 00:00:00


date              0
store_nbr         0
transactions    108
dtype: int64

In [71]:
#dates that didn't have zero transactions but are missing
transactions.transactions = transactions.groupby("store_nbr", group_keys=False).transactions.apply(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

transactions.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [72]:
transactions.shape[0] #check no rows

92016

In [73]:
df=pd.merge(df,transactions, how='left', on=['date','store_nbr'])
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,Futbol,Independencia de Cuenca,Independencia de Guayaquil,Navidad,Primer Grito de Independencia,Primer dia del ano,Terremoto Manabi,Viernes Santo,oil_price,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1,0,0,93.14,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1,0,0,93.14,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1,0,0,93.14,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1,0,0,93.14,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1,0,0,93.14,0.0


In [74]:
df=df.sort_values(by=['date', 'store_nbr']).reset_index(drop=True)
df=df.drop(columns='id')

In [75]:
df.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,...,Futbol,Independencia de Cuenca,Independencia de Guayaquil,Navidad,Primer Grito de Independencia,Primer dia del ano,Terremoto Manabi,Viernes Santo,oil_price,transactions
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,...,0,0,0,0,0,1,0,0,93.14,0.0


In [76]:
df.shape[0]

3036528

In [77]:
df.to_csv('novi_datasetovi/train_test_v2.csv')